In [2]:
import pandas as pd
import numpy as np
import s3fs
import json
from sklearn.metrics.pairwise import cosine_similarity
import datetime

In [3]:
# Import the dataframes
df_orders = pd.read_csv("datasets/olist_order_items_dataset.csv")
df_products = pd.read_csv("datasets/olist_products_dataset.csv")

In [ ]:
# Drop columns
df_orders.drop(['order_item_id', 'seller_id', 'shipping_limit_date', 'price', 'freight_value'], axis=1, inplace=True)
df_products.drop(['product_name_lenght', 'product_description_lenght', 'product_photos_qty', 'product_weight_g', 'product_length_cm', 'product_height_cm', 'product_width_cm'], axis=1, inplace=True)

In [402]:
# Merge the df
sales_df = pd.merge(left=df_orders, right=df_products, on='product_id')

In [403]:
sales_df

,order_id,product_id,product_category_name
0,00010242fe8c5a6d1ba2dd792cb16214,4244733e06e7ecb4970a6e2683c13e61,cool_stuff
1,130898c0987d1801452a8ed92a670612,4244733e06e7ecb4970a6e2683c13e61,cool_stuff
2,532ed5e14e24ae1f0d735b91524b98b9,4244733e06e7ecb4970a6e2683c13e61,cool_stuff
3,6f8c31653edb8c83e1a739408b5ff750,4244733e06e7ecb4970a6e2683c13e61,cool_stuff
4,7d19f4ef4d04461989632411b7e588b9,4244733e06e7ecb4970a6e2683c13e61,cool_stuff
...,...,...,...
112645,ffebd80e3291e811c308365936897efd,4cc4d02efc8f249c13355147fb44e34d,ferramentas_jardim
112646,ffee31fb4b5e35c9123608015637c495,b10ecf8e33aaaea419a9fa860ea80fb5,moveis_decoracao
112647,fff7c4452f050315db1b3f24d9df5fcd,dd469c03ad67e201bc2179ef077dcd48,relogios_presentes
112648,fffa82886406ccf10c7b4e35c4ff2788,bbe7651fef80287a816ead73f065fc4b,esporte_lazer


In [404]:
# Group the rows by order_id and count how many times each order_id appears
grouped = sales_df.groupby("order_id").size().reset_index(name="counts")

In [405]:
# Select only the rows where the count is greater than 1
result = grouped[grouped["counts"] > 1]

In [406]:
# Filter the original DataFrame using the selected order_id
filtered_df = sales_df[sales_df["order_id"].isin(result["order_id"])]

In [407]:
filtered_df

,order_id,product_id,product_category_name
7,c9ef97d2854afe64a3b4488bc2836af6,4244733e06e7ecb4970a6e2683c13e61,cool_stuff
11,45211f00dfa76aed1c20d6910d973222,c777355d18b72b67abbeef9df44fd0fd,moveis_decoracao
45,09268d8b25dd31ae78b464efd453d069,d63c1011f49d98b976c352955b1c4bea,beleza_saude
55,398545ae905dad286c70f01c7e7311e1,d63c1011f49d98b976c352955b1c4bea,beleza_saude
56,398545ae905dad286c70f01c7e7311e1,d63c1011f49d98b976c352955b1c4bea,beleza_saude
...,...,...,...
112634,ffd6f465cab015b452d398ac1f5d978a,fdcf45aa23bb8312ecc0027d6e1ef1c4,artigos_de_natal
112635,ffd6f465cab015b452d398ac1f5d978a,fdcf45aa23bb8312ecc0027d6e1ef1c4,artigos_de_natal
112636,ffd6f465cab015b452d398ac1f5d978a,fdcf45aa23bb8312ecc0027d6e1ef1c4,artigos_de_natal
112641,ffe4b41e99d39f0b837a239110260530,e97df839917a6e93404867b1d0319bfc,brinquedos


In [408]:
# Reset Index
filtered_df = filtered_df.reset_index(drop=True)

In [409]:
filtered_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 23787 entries, 0 to 23786
Data columns (total 3 columns):
 #   Column                 Non-Null Count  Dtype 
---  ------                 --------------  ----- 
 0   order_id               23787 non-null  object
 1   product_id             23787 non-null  object
 2   product_category_name  23471 non-null  object
dtypes: object(3)
memory usage: 557.6+ KB


In [410]:
# Pivot table where each row is an order and each column is a product and the values are the counts of the products in each of the orders
pivot_df = pd.pivot_table(filtered_df, index='order_id', columns='product_id', values='product_category_name', aggfunc='count')
pivot_df.reset_index(inplace=True)
pivot_df = pivot_df.fillna(0)
pivot_df = pivot_df.drop('order_id', axis=1)

In [411]:
pivot_df

product_id,0011c512eb256aa0dbbb544d8dffcf6e,001795ec6f1b187d37335e1c4704762e,001b72dfd63e9833e8c02742adf472e3,00210e41887c2a8ef9f791ebc780cc36,00250175f79f584c14ab5cecd80553cd,002af88741ba70c7b5cf4e4a0ad7ef85,003a31970fea14fffe92ac856b8a9b97,003c0b8f6580c850bd2e32044d2ac307,0042f1a9a7e0edd1400c6cd0fda065f8,004636c889c7c3dad6631f136b7fa082,...,ffbc83054b3741a8d67fc59d9cf9d42d,ffbe169d395060d7fb975c990581a329,ffc9caf33e2d1e9f44e3e06da19085f7,ffcfaba393e8ef71937c6e8421bc2868,ffd4bf4306745865e5692f69bd237893,ffe8083298f95571b4a66bfbc1c05524,ffef256879dbadcab7e77950f4f4a195,fff0a542c3c62682f23305214eaeaa24,fff1059cd247279f3726b7696c66e44e,fff28f91211774864a1000f918ed00cc
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9798,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9799,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9800,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9801,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [395]:
# Next we transform our pivot table into a co-occurrence matrix by taking the dot product of the pivot table and its transpose.
co_matrix = pivot_df.T.dot(pivot_df)
np.fill_diagonal(co_matrix.values, 0)

In [396]:
co_matrix

product_id,0011c512eb256aa0dbbb544d8dffcf6e,001795ec6f1b187d37335e1c4704762e,001b72dfd63e9833e8c02742adf472e3,00210e41887c2a8ef9f791ebc780cc36,00250175f79f584c14ab5cecd80553cd,002af88741ba70c7b5cf4e4a0ad7ef85,003a31970fea14fffe92ac856b8a9b97,003c0b8f6580c850bd2e32044d2ac307,0042f1a9a7e0edd1400c6cd0fda065f8,004636c889c7c3dad6631f136b7fa082,...,ffbc83054b3741a8d67fc59d9cf9d42d,ffbe169d395060d7fb975c990581a329,ffc9caf33e2d1e9f44e3e06da19085f7,ffcfaba393e8ef71937c6e8421bc2868,ffd4bf4306745865e5692f69bd237893,ffe8083298f95571b4a66bfbc1c05524,ffef256879dbadcab7e77950f4f4a195,fff0a542c3c62682f23305214eaeaa24,fff1059cd247279f3726b7696c66e44e,fff28f91211774864a1000f918ed00cc
product_id,,,,,,,,,,,,,,,,,,,,,
0011c512eb256aa0dbbb544d8dffcf6e,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
001795ec6f1b187d37335e1c4704762e,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
001b72dfd63e9833e8c02742adf472e3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
00210e41887c2a8ef9f791ebc780cc36,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
00250175f79f584c14ab5cecd80553cd,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
ffe8083298f95571b4a66bfbc1c05524,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
ffef256879dbadcab7e77950f4f4a195,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
fff0a542c3c62682f23305214eaeaa24,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [397]:
# And to transform the co-occurrence matrix into a matrix of cosine similarities between our products we utilize the cosine_similarity function from sklearn.
cos_score_df = pd.DataFrame(cosine_similarity(co_matrix))
cos_score_df.index = co_matrix.index
cos_score_df.columns = np.array(co_matrix.index)

In [398]:
cos_score_df

,0011c512eb256aa0dbbb544d8dffcf6e,001795ec6f1b187d37335e1c4704762e,001b72dfd63e9833e8c02742adf472e3,00210e41887c2a8ef9f791ebc780cc36,00250175f79f584c14ab5cecd80553cd,002af88741ba70c7b5cf4e4a0ad7ef85,003a31970fea14fffe92ac856b8a9b97,003c0b8f6580c850bd2e32044d2ac307,0042f1a9a7e0edd1400c6cd0fda065f8,004636c889c7c3dad6631f136b7fa082,...,ffbc83054b3741a8d67fc59d9cf9d42d,ffbe169d395060d7fb975c990581a329,ffc9caf33e2d1e9f44e3e06da19085f7,ffcfaba393e8ef71937c6e8421bc2868,ffd4bf4306745865e5692f69bd237893,ffe8083298f95571b4a66bfbc1c05524,ffef256879dbadcab7e77950f4f4a195,fff0a542c3c62682f23305214eaeaa24,fff1059cd247279f3726b7696c66e44e,fff28f91211774864a1000f918ed00cc
product_id,,,,,,,,,,,,,,,,,,,,,
0011c512eb256aa0dbbb544d8dffcf6e,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
001795ec6f1b187d37335e1c4704762e,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
001b72dfd63e9833e8c02742adf472e3,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
00210e41887c2a8ef9f791ebc780cc36,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
00250175f79f584c14ab5cecd80553cd,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
ffe8083298f95571b4a66bfbc1c05524,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
ffef256879dbadcab7e77950f4f4a195,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
fff0a542c3c62682f23305214eaeaa24,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [379]:
#Elimino los productos que solo se relacionan con si mismos (elimino filas y columnas)
index_to_drop = np.where((np.diag(cos_score_df) == 1) & (cos_score_df.sum(axis=1) == 1))[0]
cos_score_df = cos_score_df.drop(cos_score_df.columns[index_to_drop], axis=1)
cos_score_df = cos_score_df.drop(cos_score_df.index[index_to_drop], axis=0)


In [380]:
cos_score_df

,0011c512eb256aa0dbbb544d8dffcf6e,001795ec6f1b187d37335e1c4704762e,00210e41887c2a8ef9f791ebc780cc36,00250175f79f584c14ab5cecd80553cd,002af88741ba70c7b5cf4e4a0ad7ef85,003a31970fea14fffe92ac856b8a9b97,003c0b8f6580c850bd2e32044d2ac307,004636c889c7c3dad6631f136b7fa082,005030ef108f58b46b78116f754d8d38,007c63ae4b346920756b5adcad8095de,...,ffadc6f8aa93aadfea73b1348a058358,ffaf0af7eebb57c7f262b51ebb05dfd6,ffbbf6b9097237a1122f17e7341a3fb2,ffbe169d395060d7fb975c990581a329,ffc9caf33e2d1e9f44e3e06da19085f7,ffe8083298f95571b4a66bfbc1c05524,ffef256879dbadcab7e77950f4f4a195,fff0a542c3c62682f23305214eaeaa24,fff1059cd247279f3726b7696c66e44e,fff28f91211774864a1000f918ed00cc
product_id,,,,,,,,,,,,,,,,,,,,,
0011c512eb256aa0dbbb544d8dffcf6e,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
001795ec6f1b187d37335e1c4704762e,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
00210e41887c2a8ef9f791ebc780cc36,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
00250175f79f584c14ab5cecd80553cd,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
002af88741ba70c7b5cf4e4a0ad7ef85,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
ffe8083298f95571b4a66bfbc1c05524,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
ffef256879dbadcab7e77950f4f4a195,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
fff0a542c3c62682f23305214eaeaa24,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [289]:
cos_score_df.to_csv('co-occurrence_matrix.csv')

In [413]:
# Para un producto especifico
product = '0011c512eb256aa0dbbb544d8dffcf6e'
related_products = cos_score_df[product].sort_values(ascending=False)
related_products = related_products.drop(product)
threshold = 0.1
related_products = related_products[related_products > threshold]

In [414]:
related_products

product_id
401717fdb76bea40aad511f64ab03c92    0.666667
5a6849f9db912b68be72145d959d61d2    0.666667
cae8c962ad47ff05a508e6fec4146e1a    0.408248
22c7191da97262f0f39de80e1951dce2    0.258199
9d240cb98df035e3ef147117b92aa0e7    0.258199
e6e3140cdd035e915d6d4a9eae2ccead    0.258199
15b5d9fcf84c060e00b0883def7d10e6    0.258199
972d4aba7eb9ba03a5b7950751f11f60    0.258199
Name: 0011c512eb256aa0dbbb544d8dffcf6e, dtype: float64

In [415]:
# Para todos los productos
best_related_products = {}
for product in cos_score_df.index:
    related_products = cos_score_df[product].sort_values(ascending=False)
    related_products = related_products.drop(product)
    threshold = 0.1
    related_products = related_products[related_products > threshold]
    best_related_products[product] = related_products

In [416]:
best_related_products["0011c512eb256aa0dbbb544d8dffcf6e"]

product_id
401717fdb76bea40aad511f64ab03c92    0.666667
5a6849f9db912b68be72145d959d61d2    0.666667
cae8c962ad47ff05a508e6fec4146e1a    0.408248
22c7191da97262f0f39de80e1951dce2    0.258199
9d240cb98df035e3ef147117b92aa0e7    0.258199
e6e3140cdd035e915d6d4a9eae2ccead    0.258199
15b5d9fcf84c060e00b0883def7d10e6    0.258199
972d4aba7eb9ba03a5b7950751f11f60    0.258199
Name: 0011c512eb256aa0dbbb544d8dffcf6e, dtype: float64